In [354]:
!pip install schedule

In [380]:
import pandas as pd
# import timeit
import schedule    
import time
from datetime import datetime

import requests
from bs4 import BeautifulSoup

import fasttext
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import T5ForConditionalGeneration

In [58]:
model_class = fasttext.load_model("classification/ru_cat_v6.ftz")

In [2]:
tokenizer_resume = AutoTokenizer.from_pretrained("IlyaGusev/mbart_ru_sum_gazeta")
model_resume = AutoModelForSeq2SeqLM.from_pretrained("IlyaGusev/mbart_ru_sum_gazeta")

tokenizer_title = AutoTokenizer.from_pretrained("IlyaGusev/rut5_base_headline_gen_telegram")
model_title = T5ForConditionalGeneration.from_pretrained("IlyaGusev/rut5_base_headline_gen_telegram")

In [7]:
black_label = 'ДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РАСПРОСТРАНЕНО ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА, И (ИЛИ) РОССИЙСКИМ ЮРИДИЧЕСКИМ ЛИЦОМ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА'

In [11]:
def article2summary(article_text):
    """Делает краткое саммари из новости"""
    input_ids = tokenizer_resume(
    [article_text],
    max_length=600,
    truncation=True,
    return_tensors="pt")["input_ids"]
    
    output_ids = model_resume.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4)[0]
    
    summary = tokenizer_resume.decode(output_ids, skip_special_tokens=True)
    
    return summary  

In [4]:
def summary2title(summary):
    """Делает заголовок из краткой новости"""
    
    input_ids = tokenizer_title(
    [summary],
    max_length=600,
    add_special_tokens=True,
    padding="max_length",
    truncation=True,
    return_tensors="pt")["input_ids"]

    output_ids = model_title.generate(
    input_ids=input_ids)[0]
    
    title = tokenizer_title.decode(output_ids, skip_special_tokens=True)
    
    return title

In [384]:
def make_clean_text(article, date):
    soup = BeautifulSoup(article)
    
    first_a = soup.find('a')
    try:
        first_link = first_a.get('href')
    except AttributeError:
        first_link = 'NaN'
    
    text = soup.get_text()
    text = text.replace("\xa0", ' ').replace("\n", ' ')
    text = text.replace(black_label, '\n')
    short_news = article2summary(text)
    title = summary2title(short_news)
    
    return {'date': date, 'title': title, 'short_news': short_news, 'first_link': first_link, 'raw_news': text}

In [604]:
def make_articles_dict(channel_name):
    answer = requests.get('https://tg.i-c-a.su/json/' + channel_name)
    data = answer.json()
    messages = data['messages']
    
    temp_df = pd.read_pickle('table_news.pkl')
    start_id = temp_df.index[temp_df['agency'] == channel_name][0]  # вытаскиваем самую последнюю статью данного медиа из БД
    del(temp_df)
    
    # выбираем только те статьи, которые старше последней
    id_articles = [(el, messages[el]['id']) for el in range(len(messages)) if messages[el]['id'] > start_id]
    
    draft_articles = [make_clean_text(messages[el[0]]['message'], messages[el[0]]['date']) for el in id_articles]
    articles_dict = {el[1]: draft_articles[el[0]] for el in id_articles}
    
    # Удаляем пустые статьи для новостей, которые не содержали текст
    empty_keys = [k for k,v in articles_dict.items() if not v['raw_news']]
    for k in empty_keys:
        del articles_dict[k]
    
    return articles_dict

In [586]:
answer = requests.get('https://tg.i-c-a.su/json/' + 'meduzalive')
data = answer.json()
data

{'_': 'messages.channelMessages',
 'flags': 0,
 'inexact': False,
 'pts': 157510,
 'count': 62771,
 'messages': [{'_': 'message',
   'flags': 18048,
   'out': False,
   'mentioned': False,
   'media_unread': False,
   'silent': False,
   'post': True,
   'from_scheduled': False,
   'legacy': False,
   'edit_hide': False,
   'pinned': False,
   'noforwards': False,
   'id': 63485,
   'peer_id': {'_': 'peerChannel', 'channel_id': 1036240821},
   'date': 1656669267,
   'message': '<strong>Утренний брифинг Минобороны РФ<br />\n</strong><br />\nДАННОЕ С<a href="https://t.me/tass_agency/144718" target="_blank" rel="nofollow">ООБЩЕНИ</a>Е (МАТЕРИАЛ) СОЗДАНО\xa0И (ИЛИ) РАСПРОСТРАНЕНО ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА, И\xa0(ИЛИ) РОССИЙСКИМ ЮРИДИЧЕСКИМ ЛИЦОМ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА<br />\n<br />\n➤ВС РФ родолжают успешное наступление в\xa0районе Лисичанска. За\xa0прошедшие трое суток взяты под контроль Лисичанский нефтеперерабаты

In [590]:
type(data['messages'][0]['message'])

str

In [593]:
data['messages'][0]['date']

1656669267

In [592]:
type(data['messages'][0]['date'])

int

In [460]:
def agency2db(channel_name):
    db = pd.read_pickle('db.pkl')
    channel_dict = make_articles_dict(channel_name)
    if channel_dict:
        df = pd.DataFrame(channel_dict).T
        df['category'] = df['short_news'].apply(lambda x: model_class.predict(x)[0][0].split('__')[-1])  # классифицируем
        df = df.loc[df['category'] != 'not_news']  # удаляем новости, которые классификатор не признал новостями
        df['date'] = df['date'].apply(lambda x: datetime.fromtimestamp(x))  # преобразовываем timestamp-число в дату
        df['agency'] = channel_name
        db = pd.concat([db, df]).sort_values('date', ascending=False)
        db.to_pickle('db.pkl')

In [445]:
def join_all(agency_list):
    for agency in agency_list:
        print(f'Собираю {agency}...')
        agency2db(agency)
        print(f'....... complited')

    db = pd.read_pickle('db.pkl')
    return db

### Разбираемся с датами и периодическим парсингом

In [48]:
timeit.timeit('nplusone = record_newsbatch2db(agency_name[1])', globals=globals(), number=1)

228.33496230000037

In [51]:
228/60*len(agency_name)

45.599999999999994

In [147]:
nplusone.date.loc[17990] + pd.Timedelta("1 hour")

Timestamp('2022-06-26 09:14:01')

In [163]:
today =  pd.to_datetime('today').normalize()
start_time = today + pd.Timedelta("7 hour")
start_time

Timestamp('2022-06-29 07:00:00')

In [166]:
idx = pd.date_range(start_time, periods=4, freq="5H")
ts = pd.Series(range(len(idx)), index=idx)
ts

2022-06-29 07:00:00    0
2022-06-29 12:00:00    1
2022-06-29 17:00:00    2
2022-06-29 22:00:00    3
Freq: 5H, dtype: int64

In [173]:
len(idx), idx[0]

(4, Timestamp('2022-06-29 07:00:00', freq='5H'))

In [171]:
pd.Timestamp.now()

Timestamp('2022-06-29 08:37:33.273123')

In [353]:
df_pickle.date.iloc[0]

Timestamp('2022-06-29 15:05:30')

In [566]:
first = pd.to_datetime('today')

In [567]:
second = pd.to_datetime('today')

In [568]:
print(first, second)

2022-07-01 12:37:28.951656 2022-07-01 12:37:42.411020


In [584]:
str(pd.to_timedelta(second - first))

'0 days 00:00:13.459364'

### Запускаем парсинг

In [395]:
db.head(10)

,date,title,short_news,first_link,raw_news,category,agency
35212,2022-06-29 19:23:40,«Яндекс» подал заявление на ликвидацию структу...,«Яндекс» подал заявление на ликвидацию структу...,vc.ru/finance/452888,«Яндекс» подал заявление на ликвидацию структу...,economy,vcnews
35209,2022-06-29 18:30:11,Глава департамента предпринимательства Москвы ...,Глава департамента предпринимательства Москвы ...,https://go.the.tj/HPF6CE,Следить за новостями о новых мерах поддержки б...,economy,vcnews
35208,2022-06-29 18:15:52,Шведская компания Sandvik решила уйти из России,Минтранс заключил договоры с 20 авиакомпаниями...,vc.ru/finance/452252,Продолжаем следить за новостями: — Минтранс з...,economy,vcnews
35207,2022-06-29 18:05:25,Закон об отзыве лицензии у СМИ за «фейки» прин...,"Эксперты отмечают, что закон об отзыве лицензи...",vc.ru/legal/452811,Госдума во втором чтении одобрила законопроект...,society,vcnews
18002,2022-06-29 17:43:56,В Перуанском лесу обнаружен новый вид амфибий,В перуанском лесу обнаружен новый вид амфибий ...,https://nplus1.ru/news/2022/06/29/rhinella-una...,"Иногда чтобы открыть новый вид животных, зооло...",science,nplusone
35206,2022-06-29 16:37:31,«ВКонтакте» тестирует верификацию страниц госо...,«ВКонтакте» тестирует верификацию страниц госо...,vc.ru/social/452700,Минцифры и «ВКонтакте» тестируют верификацию с...,society,vcnews
35205,2022-06-29 16:27:11,В «ВКонтакте» распространяют анонимные посты с...,В городских пабликах во «ВКонтакте» распростра...,vc.ru/marketing/452730,В городских пабликах во «ВКонтакте» распростра...,society,vcnews
35204,2022-06-29 15:56:08,«Новые решения» получили контроль над сервисам...,«Новые решения» получили контроль над сервисам...,vc.ru/services/452662,"Новый владелец «СберЗвука», Okko, SberCloud и ...",society,vcnews
63325,2022-06-29 15:05:30,Губернатор Белгородской области перестал инфор...,Губернатор Белгородской области Вячеслав Гладк...,https://t.me/vvgladkov/500,Губернатор Белгородской области Вячеслав Гладк...,society,meduzalive
63324,2022-06-29 15:02:51,Губернатор Брянской области сообщил о сбитом у...,Губернатор Брянской области Александр Богомаз ...,https://t.me/avbogomaz/570,Губернатор Брянской области Александр Богомаз ...,society,meduzalive


In [398]:
db.agency.value_counts()

meduzalive    19
vcnews        16
TJournal      10
nplusone      10
rbc_sport     10
vedomosti     10
rozetked       8
VwordMedia     8
now_ka         8
addmeto        7
ohmypain       3
Name: agency, dtype: int64

In [399]:
db.category.value_counts()

society       55
science       16
economy       14
technology    13
sports        10
other          1
Name: category, dtype: int64

In [408]:
for el in db['title']:
    print(el)

«Яндекс» подал заявление на ликвидацию структуры в Ирландии
Глава департамента предпринимательства Москвы Алексей Фурсин в прямом эфире отвечает на вопросы предпринимателей
Шведская компания Sandvik решила уйти из России
Закон об отзыве лицензии у СМИ за «фейки» принят во втором чтении
В Перуанском лесу обнаружен новый вид амфибий
«ВКонтакте» тестирует верификацию страниц госорганизаций через Госуслуги
В «ВКонтакте» распространяют анонимные посты с рекомендацией есть кошачий корм «365 дней» на гарнир
«Новые решения» получили контроль над сервисами «СберЗвука», Okko, SberCloud и ЦРТ
Губернатор Белгородской области перестал информировать население о происшествиях на границе с Украиной
Губернатор Брянской области сообщил о сбитом украинском беспилотнике
Основатель Virgin Galactic изучил возможности реконструкции аэродрома «Антонов»
Российская авиация нанесла ракетный удар по Одесской области
Зеленский выступил на саммите НАТО с призывом «помочь Украине закончить войну победой на поле боя»

In [409]:
for el in db['short_news']:
    print(el)

«Яндекс» подал заявление на ликвидацию структуры в Ирландии, которую зарегистрировал в январе 2022 года. Так компания хотела получить налоговые льготы, рассказал источник «Коммерсанта». Но от идеи отказались — чтобы «никого не провоцировать».
Глава департамента предпринимательства Москвы Алексей Фурсин в прямом эфире отвечает на вопросы предпринимателей в телеграм-канале Алексея Фурсина. Напоминает о дедлайнах подачи заявок на гранты и отчитывается о проделанной работе. Например, в июне активно рассказывал о деловой составляющей Moscow Fashion Week.
Минтранс заключил договоры с 20 авиакомпаниями о субсидировании возврата денег пассажирам за отменённые после 24 февраля рейсы на общую сумму 16,9 млрд рублей; Шведская компания Sandvik, которая занимается инжинирингом в горнодобывающей отрасли, решила уйти из России.
Эксперты отмечают, что закон об отзыве лицензии у СМИ за «фейки» будет принят во втором чтении. Теперь это будут делать не сразу — за первое нарушение работу СМИ приостановят 

In [248]:
df.category.value_counts()

society       42
science       14
technology    12
economy       11
sports        10
other          1
Name: category, dtype: int64

In [252]:
df.agency.value_counts()

TJournal      10
rbc_sport     10
vedomosti     10
vcnews         9
meduzalive     9
rozetked       8
nplusone       8
VwordMedia     8
now_ka         8
addmeto        7
ohmypain       3
Name: agency, dtype: int64

In [345]:
db = pd.read_pickle('db.pkl')

In [413]:
del(db)

In [443]:
join_all(agency_list)

Собираю rbc_sport...
....... complited


Собираю ohmypain...
....... complited


Собираю addmeto...
....... complited




,date,title,short_news,first_link,raw_news,category,agency
63382,2022-06-30 10:25:46,Вячеслав Володин прокомментировал вступление Ф...,Вячеслав Володин прокомментировал возможное вс...,https://t.me/vv_volodin/504,Вячеслав Володин прокомментировал возможное вс...,society,meduzalive
63381,2022-06-30 10:23:57,В Беларуси сорвали праздник для детей беженцев,В Беларуси сорвали праздник для детей украинск...,https://news.zerkalo.io/life/16948.html?tg,В Беларуси сорвали праздник для детей украинск...,society,meduzalive
22885,2022-06-30 10:22:09,Мосбиржа открылась с повышением курса доллара ...,В пятницу Мосбиржа открылась с повышением курс...,tg://resolve?domain=@vedomosti,Как открылась Мосбиржа 🔻 Курс доллара на 10:1...,economy,vedomosti
12985,2022-06-30 10:04:52,Xiaomi представит новый фитнес-браслет с увели...,В июле Xiaomi представит новый фитнес-браслет ...,https://rozetked.me/news/24218-xiaomi-ob-yavil...,Это Xiaomi Smart Band 7 Pro! Презентация фитн...,technology,rozetked
63379,2022-06-30 10:04:07,"В Москве появились билборды, прославляющие уча...",В Москве и других городах России появились бил...,https://tvzvezda.ru/news/2022317134-wtzsO.html,🤦‍♂️🔍 В Москве и других городах появились билб...,society,meduzalive
...,...,...,...,...,...,...,...
5713,2022-06-20 13:31:02,Колонка Сергея Чапнина об отставке митрополита...,ging.ru – Колонка Сергея Чапнина об отставке м...,https://vpost-media.ru/opinions/za-chto-patria...,Колонка Сергея Чапнина об отставке митрополита...,society,VwordMedia
242,2022-06-17 23:33:42,Тренировка в виртуальной реальности может сниз...,Тренировка в виртуальной реальности может сниз...,https://www.tohoku.ac.jp/en/press/training_vir...,🔋🧠💻 Тренировка в виртуальной реальности может ...,science,now_ka
241,2022-06-15 23:05:59,Ученые обнаружили молодую планету с массой Юпи...,"Спутники далеких звезд Мозаика пыльных, вращаю...",https://noirlab.edu/public/news/noirlab2212/,🛸 Изображения с телескопа Gemini South в Чили ...,science,now_ka
239,2022-06-13 23:50:58,Ученые нашли новый способ лечения хронической ...,молекула в нервной системе может быть ключом к...,https://ucalgary.ca/news/ucalgary-researchers-...,💊🧠 Молекула в нервной системе может быть ключо...,science,now_ka


In [463]:
db.category.value_counts()

society          78
economy          22
technology       21
science          16
sports           11
entertainment     6
other             1
Name: category, dtype: int64

In [464]:
db.agency.value_counts()

meduzalive    27
vcnews        23
vedomosti     19
TJournal      18
rbc_sport     14
rozetked      12
VwordMedia    11
nplusone      10
addmeto        9
now_ka         8
ohmypain       4
Name: agency, dtype: int64

In [465]:
agencies = [
    'vcnews',
    'nplusone',
    'TJournal',
    'vedomosti',
    'VwordMedia',
    'meduzalive',
    'rozetked',
    'now_ka',
    'rbc_sport',
    'ohmypain', 
    'addmeto']

In [468]:
# schedule.every().day.at("07:00").do(join_all, agency_list = agencies)
# schedule.every().day.at("13:05").do(join_all, agency_list = agencies)
# schedule.every().day.at("17:12").do(join_all, agency_list = agencies)
# schedule.every().day.at("21:00").do(join_all, agency_list = agencies)

# while True:
#     schedule.run_pending()
#     time.sleep(1)

In [477]:
db.to_csv('db.csv', encoding='utf-8')

In [509]:
db = pd.read_pickle('db.pkl')

In [508]:
schedule.clear()

In [560]:
db.index.names = ['id_news']

In [511]:
db.category.value_counts()

society          147
economy           44
technology        36
sports            20
science           17
entertainment     13
other              1
Name: category, dtype: int64

In [512]:
db.short_news[db.category != 'society']

12993    «Обидно видеть то, что не можешь купить, правд...
63455    «Беспорно» — фильм-исповедь подростка, который...
63454    В Гоголь-центре из Авиньона прошла премьера сп...
63449    Volkswagen зарабатывает «так много, как никогд...
22918    Сбербанк повысит зарплаты сотрудникам на 8,5%....
                               ...                        
5714     Политолог Борис Минаев – о Геннадии Бурбулисе,...
242      Тренировка в виртуальной реальности может сниз...
241      Спутники далеких звезд Мозаика пыльных, вращаю...
239      молекула в нервной системе может быть ключом к...
238      На острове Уайт обнаружен крупнейший в Европе ...
Name: short_news, Length: 131, dtype: object

In [553]:
length = db.category.to_list()

In [556]:
result = max(len(x) for x in length)
result

13

In [621]:
schedule.clear()

In [599]:
db.agency.value_counts()

meduzalive      55
vedomosti       46
vcnews          44
TJournal        31
rbc_sport       22
rozetked        19
VwordMedia      14
nplusone        13
addmeto         11
russianmacro    10
now_ka           8
ohmypain         5
Name: agency, dtype: int64

In [601]:
db[db.agency == 'russianmacro']

,date,title,short_news,first_link,raw_news,category,agency
id_news,,,,,,,
14884,2022-06-30 18:30:10,ЦБ увеличил лимит перевода средств физлиц на с...,ЦБ увеличил лимит перевода средств физлиц на с...,http://cbr.ru/press/event/?id=13976,ЦБ ПРОДОЛЖАЕТ ПОПЫТКИ ВЫТОЛКАТЬ ВАЛЮТУ ИЗ СТРА...,economy,russianmacro
14883,2022-06-30 15:08:22,В мае грузооборот транспорта в РФ снизился на ...,В мае грузооборот транспорта в РФ снизился на ...,NaN,ГРУЗООБОРОТ В МАЕ: ТРУБОПРОВОДНЫЙ ТРАНСПОРТ НЕ...,economy,russianmacro
14882,2022-06-30 14:30:05,Российский ипотечный рынок пережил коллапс,Российский ипотечный рынок пережил коллапс. В ...,NaN,"КОЛЛАПС ИПОТЕЧНОГО РЫНКА, НАЧАВШИЙСЯ В АПРЕЛЕ,...",economy,russianmacro
14881,2022-06-30 12:31:58,"Российские санкции, отключение России от SWIFT...","Российские санкции, отключение России от SWIFT...",tg://resolve?domain=@e_magic,⚡️В связи со сложившейся геополитической ситуа...,economy,russianmacro
14880,2022-06-30 12:03:42,Российский фондовый рынок – это гиблое место д...,Российский фондовый рынок – это сейчас гиблое ...,https://www.gazprom.ru/press/news/2022/june/ar...,ЕСТЬ ВЕЩИ ПОВАЖНЕЕ ФОНДОВОГО РЫНКА После того...,economy,russianmacro
14879,2022-06-30 11:30:49,Росстат зафиксировал замедление промышленного ...,Росстат зафиксировал замедление промышленного ...,https://t.me/russianmacro/14865,ПАДЕНИЕ ПРОМЫШЛЕННОСТИ В МАЕ ЗАМЕДЛИЛОСЬ. ВЫПУ...,economy,russianmacro
14878,2022-06-30 10:15:24,Эксперты «Газпромбанка.Мнение» обновляют прогн...,Эксперты «Газпромбанка.Мнение» обновляют прогн...,https://telegra.ph/Svezhie-makroprognozy-06-30,💡Сегодня в еженедельной рубрике «Газпромбанк.М...,economy,russianmacro
14877,2022-06-30 10:01:52,Инфляция во Франции продолжит расти,На фоне ускорения цен на энергоносители инфляц...,https://t.me/russianmacro/14870,ИНФЛЯЦИЯ ВО ФРАНЦИИ: ТЕМПЫ ПРОДОЛЖАЮТ УВЕРЕННО...,economy,russianmacro
14876,2022-06-30 09:12:03,Китайские индексы торгуются в плюсе на хороших...,"Снижение ВВП США в 1кв22, с -1.5% до -1.6% анн...",https://t.me/russianmacro/14875,🔴🟢 - минимальные ценовые изменения на WallStre...,economy,russianmacro


In [602]:
def ag(channel_name: str) -> None:
	"""Получает словарь текущего СМИ, и пишет его в нашу базу данных, предварительно обработав и отсортировав"""
	db = pd.read_pickle('table_news.pkl')
	channel_dict = make_articles_dict(channel_name)
	if channel_dict:
		df = pd.DataFrame(channel_dict).T
		df['category'] = df['short_news'].apply(
			lambda x: model_class.predict(x)[0][0].split('__')[-1])  # классифицируем fasttext-ом, достаём класс
		df = df.loc[df['category'] != 'not_news']  # удаляем новости, которые классификатор не признал новостями
		df['date'] = df['date'].apply(lambda x: datetime.fromtimestamp(x))  # преобразовываем timestamp-число в дату
		df['agency'] = channel_name
		ch_news = pd.concat([db, df]).sort_values('date', ascending=False)
		ch_news.to_pickle('table_news.pkl')

In [609]:
def make_articles_dict(channel_name):
    answer = requests.get('https://tg.i-c-a.su/json/' + channel_name)
    data = answer.json()
    messages = data['messages']
    
#     temp_df = pd.read_pickle('table_news.pkl')
    start_id = 0  # вытаскиваем самую последнюю статью данного медиа из БД
#     del(temp_df)
    
    # выбираем только те статьи, которые старше последней
    id_articles = [(el, messages[el]['id']) for el in range(len(messages)) if messages[el]['id'] > start_id]
    
    draft_articles = [make_clean_text(messages[el[0]]['message'], messages[el[0]]['date']) for el in id_articles]
    articles_dict = {el[1]: draft_articles[el[0]] for el in id_articles}
    
    # Удаляем пустые статьи для новостей, которые не содержали текст
    empty_keys = [k for k,v in articles_dict.items() if not v['raw_news']]
    for k in empty_keys:
        del articles_dict[k]
    
    return articles_dict

In [616]:
meduzapro = make_articles_dict('meduzapro')

In [617]:
channel_dict = meduzapro

In [618]:
db = pd.read_pickle('table_news.pkl')
df = pd.DataFrame(channel_dict).T
df['category'] = df['short_news'].apply(
    lambda x: model_class.predict(x)[0][0].split('__')[-1])  # классифицируем fasttext-ом, достаём класс
df = df.loc[df['category'] != 'not_news']  # удаляем новости, которые классификатор не признал новостями
df['date'] = df['date'].apply(lambda x: datetime.fromtimestamp(x))  # преобразовываем timestamp-число в дату
df['agency'] = 'meduzapro'
ch_news = pd.concat([db, df]).sort_values('date', ascending=False)
ch_news.to_pickle('table_news.pkl')

In [3]:
import pandas as pd

In [4]:
db = pd.read_pickle('table_news.pkl')

In [5]:
db

,date,title,short_news,first_link,raw_news,category,agency
22966,2022-07-03 12:00:54,Где в Подмосковье отдохнуть с палаткой: в Пушк...,⛺️ Где в Подмосковье отдохнуть с палаткой: в П...,http://vdmsti.ru/gurb,⛺️ Где в Подмосковье отдохнуть с палаткой Новы...,society,vedomosti
18843,2022-07-03 11:57:40,Российская сторона нанесла удар по Белгороду р...,Российская сторона нанесла удар по Белгороду р...,https://www.interfax.ru/russia/850182,"По сообщению российского Минобороны, украинска...",society,thebell_io
57262,2022-07-03 11:56:53,Мирзиёев второй раз за сутки вылетел в Каракал...,Президент Узбекистана Шавкат Мирзиёев пообещал...,https://tjournal.ru/news/668194,Президент Узбекистана прибыл в Каракалпакстан ...,society,TJournal
6474,2022-07-03 11:55:09,Минобороны РФ обвинило Украину в ударе по Белг...,Минобороны РФ обвинило Украину в ударе по Белг...,https://meduza.io/news/2022/07/03/minoborony-r...,ДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РА...,society,meduzapro
63625,2022-07-03 11:54:19,Российское Минобороны: Белгород был обстрелян ...,"Российское Минобороны утверждает, что Белгород...",NaN,"Российское Минобороны утверждает, что Белгород...",society,meduzalive
...,...,...,...,...,...,...,...
5713,2022-06-20 13:31:02,Колонка Сергея Чапнина об отставке митрополита...,ging.ru – Колонка Сергея Чапнина об отставке м...,https://vpost-media.ru/opinions/za-chto-patria...,Колонка Сергея Чапнина об отставке митрополита...,society,VwordMedia
242,2022-06-17 23:33:42,Тренировка в виртуальной реальности может сниз...,Тренировка в виртуальной реальности может сниз...,https://www.tohoku.ac.jp/en/press/training_vir...,🔋🧠💻 Тренировка в виртуальной реальности может ...,science,now_ka
241,2022-06-15 23:05:59,Ученые обнаружили молодую планету с массой Юпи...,"Спутники далеких звезд Мозаика пыльных, вращаю...",https://noirlab.edu/public/news/noirlab2212/,🛸 Изображения с телескопа Gemini South в Чили ...,science,now_ka
239,2022-06-13 23:50:58,Ученые нашли новый способ лечения хронической ...,молекула в нервной системе может быть ключом к...,https://ucalgary.ca/news/ucalgary-researchers-...,💊🧠 Молекула в нервной системе может быть ключо...,science,now_ka


In [6]:
start_id = db.index[db['agency'] == 'nltk'][0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [622]:
# schedule.clear()